# Nuclio - Training function

## Environment

In [1]:
# nuclio: ignore
import nuclio

### Configurations

In [2]:
%%nuclio config

# Trigger
spec.triggers.retrain.kind = "cron"
spec.triggers.retrain.attributes.interval = "1h"

# Base image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.retrain.kind to 'cron'
%nuclio: setting spec.triggers.retrain.attributes.interval to '1h'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Variables

In [3]:
%%nuclio env

# DB Config
V3IO_FRAMESD=${V3IO_FRAMESD}
V3IO_USERNAME=${V3IO_USERNAME}
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Features
FEATURES_TABLE=netops_features

# Predictions
PREDICTIONS_TABLE=netops_predictions

# Training
TRAIN_ON_LAST=7d
TRAIN_SIZE=0.7

# Parallelizem
NUMBER_OF_SHARDS=4

# Model
# MODEL_FILENAME=netops.model.pickle
MODEL_FILENAME=netops.v3.model.pickle
SOURCE_MODEL_DIR=/bigdata/netops/models
FIXED_WEB_DIR=/models

%nuclio: setting 'V3IO_FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'FEATURES_TABLE' environment variable
%nuclio: setting 'PREDICTIONS_TABLE' environment variable
%nuclio: setting 'TRAIN_ON_LAST' environment variable
%nuclio: setting 'TRAIN_SIZE' environment variable
%nuclio: setting 'NUMBER_OF_SHARDS' environment variable
%nuclio: setting '# MODEL_FILENAME' environment variable
%nuclio: setting 'MODEL_FILENAME' environment variable
%nuclio: setting 'SOURCE_MODEL_DIR' environment variable
%nuclio: setting 'FIXED_WEB_DIR' environment variable


%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line


In [4]:
%nuclio env -c CURRENT_MODEL_DIR=/models
%nuclio env -l CURRENT_MODEL_DIR=/v3io/bigdata/netops/models

%nuclio: setting 'CURRENT_MODEL_DIR' environment variable


### Commands

In [120]:
%%nuclio cmd

############
# installs #
############

# Utils
pip install pyarrow
pip install pandas

# Igz DB
pip install v3io_frames --upgrade

# Function
pip install xgboost
pip install scikit-learn==0.20.1

apt-get update && apt-get install -y wget
mkdir -p ${FIXED_WEB_DIR}

Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Requirement already up-to-date: v3io_frames in /User/.pythonlibs/lib/python3.6/site-packages (0.5.3)
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
mkdir: cannot create directory '/models': Permission denied


In [5]:
%%nuclio cmd -c 
# Copy the model file into the function
wget -O ${FIXED_WEB_DIR}/${MODEL_FILENAME} --header "x-v3io-session-key: ${V3IO_ACCESS_KEY}" http://${V3IO_WEBAPI_SERVICE_HOST}:8081${SOURCE_MODEL_DIR}/${MODEL_FILENAME}

## Function

### Imports

In [31]:
import os
import pickle

import v3io_frames as v3f

import pandas as pd
import xgboost as xgb

### Helper functions

In [32]:
def get_data(context):
    # Get data from DB
    df = context.v3f.read(backend='tsdb', query=f'select * from {context.features_table}',
                          start=f'now-1h', end='now', multi_index=True)
    # Drop is_error columns if available
    keep_columns = [col for col in df.columns if 'is_error' not in col]
    
    # Keep good columns and Sort them
    df = df[sorted(keep_columns)]
    return df

### Init context

In [33]:
def init_context(context):
    
    # Create our DB client
    v3io_client = v3f.Client(address='http://' + os.getenv('V3IO_FRAMESD', 'framesd:8081'), 
                        container='bigdata', 
                        password=os.environ['V3IO_ACCESS_KEY'], 
                        user=os.environ['V3IO_USERNAME'])
    setattr(context, 'v3f', v3io_client)
    
    # Save features directory
    features_table = os.getenv('FEATURES_TABLE', 'netops_features')
    setattr(context, 'features_table', features_table)
    
    # Save predictions directory
    predictions_table = os.getenv('PREDICTIONS_TABLE', 'netops_predictions')
    setattr(context, 'predictions_table', predictions_table)
    
    # Create predictions TSDB table if needed
    context.v3f.create('tsdb', context.predictions_table, attrs={'rate': '1/s'}, if_exists=1)
    
    # Load the model
    model_path = os.path.join(os.getenv('CURRENT_MODEL_DIR', '/models'), os.getenv('MODEL_FILENAME', 'netops.v1.model'))
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    setattr(context, 'model', model)

### Handler

In [34]:
def handler(context, event):

    # Load last hour data
    df = get_data(context)
    
    # limit for testing
    df = df.head(2)
    
    # Predict
    df['prediction'] = context.model.predict(df.values)
    
    # Prepare to save predictions
    df = df.reset_index()
    df = df.rename({'level_0': 'time',
                    'level_1': 'company',
                    'level_2': 'data_center',
                    'level_3': 'device'}, axis=1)
    
    # Save
    context.v3f.write(backend='tsdb', table=context.predictions_table, dfs=[df], index_cols=['time', 'data_center', 'device', 'company'])

## Test

In [35]:
# nuclio: ignore
init_context(context)

In [36]:
# nuclio: ignore
event = nuclio.Event(body='')
output = handler(context, event)
output

## Deployment

In [26]:
%nuclio deploy -p netops -n predict -c

%nuclio: ['deploy', '-p', 'netops', '-n', 'predict', '-c', '/User/netops/tutorials/demos/netops/Nuclio-Inference.ipynb']
%nuclio: [nuclio.deploy] 2019-05-19 13:40:25,175 (info) Building processor image
%nuclio: [nuclio.deploy] 2019-05-19 13:40:31,369 (info) Pushing image
%nuclio: [nuclio.deploy] 2019-05-19 13:40:31,370 (info) Build complete
%nuclio: [nuclio.deploy] 2019-05-19 13:40:51,575 (info) Function deploy complete
%nuclio: [nuclio.deploy] 2019-05-19 13:40:51,582 done updating predict, function address: 3.122.56.83:32642
%nuclio: function deployed
